<a href="https://colab.research.google.com/github/SomaKorada07/END2.0/blob/main/Session%205/Session_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>